In [ ]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from video_block import * 
from inflator import *
from triplet_loss import *

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

/home/adrian/anaconda3/envs/VE/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
#export 

vid_paths = get_files('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos', extensions='.mp4')
lbls = L(map(parent_label,vid_paths))
             
tfms = [[create_video_tensor],
        [parent_label, Categorize()]]

splits = RandomSplitter(valid_pct=0.2, seed=42)(vid_paths)
n_lbl, n_el = 4, 8
splits = [uniformize_dataset(idxs, lbls[idxs] , n_lbl=n_lbl, n_el=n_el) for idxs in splits]

dsets = Datasets(vid_paths, tfms, splits=splits)

In [ ]:
    T.RandomCrop(224),
    T.RandomHorizontalFlip(p=0.3),

In [4]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_charades.ipynb.
Converted 05_ucf.ipynb.
Converted 06_distributed_inflated_NN.ipynb.
Converted 07_read_video_function.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.


In [ ]:
# default_exp video_block